In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName('OlistData') \
.getOrCreate()

VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
2,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
hdfs_path = 's3://kalan-emr-tutorial/input/hadoop/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
order_item_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv',header=True,inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Caching Frequently Used Data for better performance

orders_df.cache()
customers_df.cache()
order_item_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

# Optimized Joins

In [5]:
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
orders_item_joined_df = orders_df.join(order_item_df, 'order_id','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
orders_items_products_df = orders_item_joined_df.join(products_df,'product_id','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
orders_items_products_sellers_df = orders_items_products_df.join(broadcast(sellers_df),'seller_id','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
full_orders_df = orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Geolocation Data

full_orders_df = full_orders_df.join(broadcast(geolocation_df), full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix, 'left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
full_orders_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+-------------------+-----+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+----------------------+-----------+------------+--------------------+------------------------+-------------+--------------+---------------------------+-------------------+------------------+----------------+-----------------+
|         customer_id|           seller_id|          product_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|shipping_limit_date|price|freight_value|product_category_name|product_name_lenght|pr

In [12]:
full_orders_df = full_orders_df.join(broadcast(reviews_df),'order_id','left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
full_orders_df = full_orders_df.join(payments_df, 'order_id', 'left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
full_orders_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [15]:
# Total Revenues Per Seller

seller_revenue_df = full_orders_df.groupBy('seller_id').agg({'price': 'sum'}).withColumnRenamed('sum(price)', 'total_revenue')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
seller_revenue_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+
|           seller_id|      total_revenue|
+--------------------+-------------------+
|8e6cc767478edae94...| 1145757.4000000008|
|cb5ff1b9715e99589...|            13235.0|
|acadd4d36859671cb...|           262034.0|
|b76dba6c951ab00dc...| 302582.65999999875|
|7a67c85e85bb2ce85...|2.031279488999999E7|
+--------------------+-------------------+
only showing top 5 rows

In [17]:
# Total Orders Per customer
customer_order_count_df = full_orders_df.groupBy('customer_id')\
.agg(count('order_id').alias('total_orders'))\
.orderBy(desc('total_orders'))

customer_order_count_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+
|         customer_id|total_orders|
+--------------------+------------+
|351e40989da90e704...|       11427|
|50920f8cd0681fd86...|       10752|
|9b43e2a62de9bab3a...|        8556|
|270c23a11d024a44c...|        8001|
|5c87184371002d49e...|        6876|
+--------------------+------------+
only showing top 5 rows

In [18]:
# Average Review Score Per Seller

seller_review_df = full_orders_df.groupBy('seller_id')\
.agg(avg('review_score').alias('avg_review_score'))\
.orderBy(desc('avg_review_score'))

seller_review_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+
|           seller_id|avg_review_score|
+--------------------+----------------+
|9c1c0c36cd23c2089...|             5.0|
|05ca864204d09595a...|             5.0|
|dcb7367a92f5ffb89...|             5.0|
|e9874ae10b2d60d43...|             5.0|
|7ab0dd5487bab2dc8...|             5.0|
+--------------------+----------------+
only showing top 5 rows

In [19]:
# Top 10 Most Sold Products

top_products_df = full_orders_df.groupBy('product_id')\
.agg(count('order_id').alias('total_sold'))\
.orderBy(desc('total_sold'))\
.limit(10)

top_products_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|          product_id|total_sold|
+--------------------+----------+
|aca2eb7d00ea1a7b8...|     86740|
|422879e10f4668299...|     81110|
|99a4788cb24856965...|     78775|
|389d119b48cf3043d...|     60248|
|d1c427060a0f73f6b...|     59274|
|368c6c730842d7801...|     58358|
|53759a2ecddad2bb8...|     52654|
|53b36df67ebb7c415...|     52105|
|154e7e31ebfa09220...|     42700|
|3dd2a17168ec895c7...|     40787|
+--------------------+----------+

In [20]:
# Top 10 Customer By Spending
top_customers_spending_df = full_orders_df.groupBy('customer_id')\
    .agg(sum('price').alias('total_spending'))\
    .orderBy(desc('total_spending'))\
    .limit(10)

top_customers_spending_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|         customer_id|    total_spending|
+--------------------+------------------+
|d3e82ccec3cb5f956...|         6662844.0|
|df55c14d1476a9a34...|         3565657.0|
|fe5113a38e3575c04...|         3293604.0|
|ec5b2ba62e5743423...|         2556120.0|
|63b964e79dee32a35...|         2501664.0|
|46bb3c0b1a65c8399...|         2336752.0|
|05455dfa7cd02f13d...| 2160194.400000087|
|3690e975641f01bd0...|         2124498.0|
|349509b216bd5ec11...|         1923627.0|
|695476b5848d64ba0...|1820543.1299999943|
+--------------------+------------------+

# Windows Function


In [21]:
from pyspark.sql.window import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Rank Top Selling Products Per Seller
# Dense Rank for Sellers based on Revenue

window_spec = Window.partitionBy('seller_id').orderBy(desc('price'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Rank Top Selling Products Per Seller

top_seller_products_df = full_orders_df.withColumn('rank',rank().over(window_spec))

top_seller_products_df.select('seller_id','price','rank').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+----+
|           seller_id|price|rank|
+--------------------+-----+----+
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
+--------------------+-----+----+
only showing top 20 rows

# Advance Aggregation and Enrichment

In [26]:
# Total Revenue & Average Order Value (AOV) per Customer

customer_spending_df = full_orders_df.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_spending'),
    round(avg('price'),2).alias('aov')
)\
.orderBy(desc('total_spending'))

customer_spending_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+------+
|         customer_id|total_orders|total_spending|   aov|
+--------------------+------------+--------------+------+
|d3e82ccec3cb5f956...|        6876|     6662844.0| 969.0|
|df55c14d1476a9a34...|         743|     3565657.0|4799.0|
|fe5113a38e3575c04...|        2292|     3293604.0|1437.0|
|ec5b2ba62e5743423...|        1428|     2556120.0|1790.0|
|63b964e79dee32a35...|        6072|     2501664.0| 412.0|
+--------------------+------------+--------------+------+
only showing top 5 rows

In [27]:
6662844/6876

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

969.0

In [28]:
# Seller Performance Metrics (Revenue, Average Review, Order Count)

seller_performance_df = full_orders_df.groupBy('seller_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(stddev('price'), 2).alias('aov'),
    round(avg('review_score'), 2).alias('avg_review_score')
)\
.orderBy(desc('total_revenue'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
seller_performance_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------------+------+----------------+
|           seller_id|total_orders|       total_revenue|   aov|avg_review_score|
+--------------------+------------+--------------------+------+----------------+
|4869f7a5dfa277a7d...|      184587| 3.613871732000001E7|111.65|            4.09|
|53243585a1d6dc264...|       54514| 3.429159294999997E7|499.65|            4.12|
|4a3ca9315b744ce9f...|      330661|3.3759570839999996E7| 59.37|            3.77|
|7c67e1448b00f6e96...|      233306|3.2282321789999586E7| 50.39|            3.42|
|fa1c13f2614d7b5c4...|       87686|3.0139386310000043E7| 307.7|            4.38|
+--------------------+------------+--------------------+------+----------------+
only showing top 5 rows

In [31]:
# Product Popularity Metrics

product_metrics_df = full_orders_df.groupBy('product_id')\
.agg(
    count('order_id').alias('total_sales'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('avg_price'),
    round(stddev('price'), 2).alias('price_volatility'),
    collect_set('seller_id').alias('unique_sellers')
)\
.orderBy(desc('total_sales'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
product_metrics_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+------------------+---------+----------------+--------------------+
|          product_id|total_sales|     total_revenue|avg_price|price_volatility|      unique_sellers|
+--------------------+-----------+------------------+---------+----------------+--------------------+
|aca2eb7d00ea1a7b8...|      86740|         6164630.3|    71.07|            3.17|[955fee9216a65b61...|
|422879e10f4668299...|      81110| 4442791.510000017|    54.77|            4.46|[1f50f920176fa81d...|
|99a4788cb24856965...|      78775| 6921762.710000004|    87.87|            4.08|[4a3ca9315b744ce9...|
|389d119b48cf3043d...|      60248|3280533.1300000064|    54.45|            4.37|[1f50f920176fa81d...|
|d1c427060a0f73f6b...|      59274|        8220103.33|   138.68|           16.58|[a1043bafd471dff5...|
+--------------------+-----------+------------------+---------+----------------+--------------------+
only showing top 5 rows

In [33]:
# Monthly Revenue and Order count trend
monthly_trend_df = full_orders_df.withColumn('year_month', date_format('order_purchase_timestamp','yyyy-MM'))\
    .groupBy('year_month')\
    .agg(
        sum('price').alias('monthly_revenue'),
        count('order_id').alias('order_count')
    
    )\
    .orderBy('year_month')
monthly_trend_df.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+-----------+
|year_month|     monthly_revenue|order_count|
+----------+--------------------+-----------+
|   2016-09|   57430.99999999988|       1268|
|   2016-10|   8301215.429999983|      62812|
|   2016-12|  3313.6000000000176|        304|
|   2017-01|1.7495019379999913E7|     154814|
|   2017-02| 3.771518491999999E7|     314901|
|   2017-03| 6.410433590999994E7|     493268|
|   2017-04| 5.830563201000004E7|     428106|
|   2017-05|  8.32050953800006E7|     681522|
|   2017-06| 6.957582136999993E7|     578515|
|   2017-07| 8.398955144000003E7|     760829|
|   2017-08| 9.661791376000051E7|     801394|
|   2017-09| 9.585077286000088E7|     761877|
|   2017-10|1.0522491751000106E8|     860156|
|   2017-11|1.6603721754000142E8|    1422336|
|   2017-12|1.2601966210000123E8|    1022973|
|   2018-01|1.5403788212000307E8|    1340766|
|   2018-02| 1.410665991500021E8|    1236262|
|   2018-03| 1.545768325200017E8|    1316199|
|   2018-04|1.5885133712000293E8| 

In [43]:
# Customer Retention Analysis (First & Last Order )

customer_retention_df = full_orders_df.groupBy('customer_id')\
.agg(
    min('order_purchase_timestamp').alias('first_order_date'),
    max('order_purchase_timestamp').alias('last_order_date'),
    count('order_id').alias('total_orders'),
    round(avg('price'),2).alias('aov')
)\
.orderBy(desc('total_orders'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
customer_retention_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-------------------+------------+------+
|         customer_id|   first_order_date|    last_order_date|total_orders|   aov|
+--------------------+-------------------+-------------------+------------+------+
|351e40989da90e704...|2017-07-13 10:42:37|2017-07-13 10:42:37|       11427| 85.99|
|50920f8cd0681fd86...|2018-01-27 11:28:32|2018-01-27 11:28:32|       10752| 43.82|
|9b43e2a62de9bab3a...|2017-05-25 22:27:50|2017-05-25 22:27:50|        8556|  26.4|
|270c23a11d024a44c...|2017-08-08 20:26:31|2017-08-08 20:26:31|        8001| 36.59|
|5c87184371002d49e...|2018-01-05 19:15:37|2018-01-05 19:15:37|        6876| 12.49|
|d3e82ccec3cb5f956...|2017-03-18 14:28:34|2017-03-18 14:28:34|        6876| 969.0|
|d5f2b3f597c7ccafb...|2017-12-13 14:21:15|2017-12-13 14:21:15|        6706|  59.0|
|c2f18647725395af4...|2018-03-06 19:21:47|2018-03-06 19:21:47|        6612|  34.9|
|24e7dc2ff8c071263...|2017-11-24 16:16:45|2017-11-24 16:16:45|        6597|  59.2|
|7bb

In [41]:
full_orders_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [42]:
from pyspark.sql.functions import col, to_timestamp
# Cast order_purchase_timestamp to timestamp
full_orders_df = full_orders_df.withColumn(
    'order_purchase_timestamp',
    to_timestamp(col('order_purchase_timestamp'), 'yyyy-MM-dd HH:mm:ss')
)
# Verify the schema again after casting
full_orders_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [45]:
# Filter for a specific customer and show their order_purchase_timestamp values
full_orders_df.filter(full_orders_df.customer_id == '351e4a089d9a307e704')\
    .select('order_purchase_timestamp')\
    .show(20, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|order_purchase_timestamp|
+------------------------+
+------------------------+